In [1]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
import time
import re
from scipy import stats
import sys
from scipy import optimize

In [ ]:
# 服占比修正函数

def rt_fzb_modified(x, x_min, x_max):
    if x <= x_min:
        return x_min
    elif x >= x_max:
        return x_max
    elif (x > x_min) & (x < x_max):
        return x
    else:
        return 0
# 分段函数


def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0, x >= x0], [lambda x:k1*x+y0-k1*x0, lambda x:k2*x+y0-k2*x0])
# 拟合函数&最优供需比修正


def func_fit(data, i, city_id, is_weekend):
    temp = data[data.hour == i]
    # step1:分段线性拟合 成交率~供需比，找拟合拐点为最优供需比
    p, e = optimize.curve_fit(piecewise_linear, np.array(
        temp['供需比分组']), np.array(temp['成交率']))
    # 供需比范围
    m_min = min(temp['供需比分组'])
    m_max = max(temp['供需比分组'])
    # step2:供需比修正
    # 1.若拐点在范围内
    if (p[0] > m_min) & (p[0] < m_max):
        # 1.1满足两个条件：a.拟合函数第一段斜率>第二段；b.拐点供需比在约束区间内
        if (p[2] > p[3]) & (p[0] >= temp['xgb_target_min'].mean()) & (p[0] <= temp['xgb_target_max'].mean()):

            xgb_target = p[0]
        # 1.2不满足两个条件
        else:
            # 1.2.1若不满足条件2（拐点小于约束区间）,则取下界
            if p[0] < temp['xgb_target_min'].mean():
                xgb_target = temp['xgb_target_min'].mean()
            # 1.2.2拐点大于约束区间,则取上界
            elif p[0] > temp['xgb_target_max'].mean():
                xgb_target = temp['xgb_target_max'].mean()
            # 1.2.3若拐点在约束区间内，但斜率不满足条件1，则用前后的均值替代(先赋-1，后面再处理)
            else:
                xgb_target = -1
    # 2.若拐点不在范围内（先赋-1，后面用前后均值处理）
    else:
        xgb_target = -1
    return p, m_min, m_max, xgb_target, temp['xgb_target_min'].mean(), temp['xgb_target_max'].mean()

    # 分城市拟合结果


def func_result(data, city_id):
    result = pd.DataFrame(columns=['city_id', 'is_weekend', 'hour', 'xgb_piecewise', 'xgb_min',
                          'xgb_max', 'cr_piecewise', 'xgb_target', 'xgb_target_min', 'xgb_target_max'])
    # 工作日
    data_workday = data[data.is_weekend == 0]
    for i in range(0, 24):
        try:
            p, m_min, m_max, xgb_target, xgb_target_min, xgb_target_max = func_fit(
                data_workday, i, city_id, '工作日')
            result = result.append({'city_id': city_id, 'is_weekend': 0, 'hour': i, 'xgb_piecewise': p[0], 'xgb_min': m_min, 'xgb_max': m_max, 'cr_piecewise': p[
                                   1], 'xgb_target': xgb_target, 'xgb_target_min': xgb_target_min, 'xgb_target_max': xgb_target_max}, ignore_index=True)
        except:
            temp = data_workday[data_workday.hour == i]
            if len(temp['供需比分组']) == 0:
                result = result.append({'city_id': city_id, 'is_weekend': 0, 'hour': i, 'xgb_piecewise': -1, 'xgb_min': -1, 'xgb_max': -1,
                                       'cr_piecewise': -1, 'xgb_target': xgb_target, 'xgb_target_min': -1, 'xgb_target_max': -1}, ignore_index=True)
                result_error = result_error.append(
                    {'city_id': city_id, 'is_weekend': 0, 'hour': i, 'error_reason': '数据缺失'}, ignore_index=True)
            else:
                result = result.append({'city_id': city_id, 'is_weekend': 0, 'hour': i, 'xgb_piecewise': -1, 'xgb_min': -1, 'xgb_max': -1,
                                       'cr_piecewise': -1, 'xgb_target': xgb_target, 'xgb_target_min': -1, 'xgb_target_max': -1}, ignore_index=True)
                result_error = result_error.append(
                    {'city_id': city_id, 'is_weekend': 0, 'hour': i, 'error_reason': '拟合失败'}, ignore_index=True)
    # 周末
    data_weekend = data[data.is_weekend == 1]
    for i in range(0, 24):
        try:
            p, m_min, m_max, xgb_target, xgb_target_min, xgb_target_max = func_fit(
                data_weekend, i, city_id, '周末')
            result = result.append({'city_id': city_id, 'is_weekend': 1, 'hour': i, 'xgb_piecewise': p[0], 'xgb_min': m_min, 'xgb_max': m_max, 'cr_piecewise': p[
                                   1], 'xgb_target': xgb_target, 'xgb_target_min': xgb_target_min, 'xgb_target_max': xgb_target_max}, ignore_index=True)
        except:
            temp = data_weekend[data_weekend.hour == i]
            if len(temp['供需比分组']) == 0:
                result = result.append({'city_id': city_id, 'is_weekend': 1, 'hour': i, 'xgb_piecewise': -1, 'xgb_min': -1, 'xgb_max': -1,
                                       'cr_piecewise': -1, 'xgb_target': xgb_target, 'xgb_target_min': -1, 'xgb_target_max': -1}, ignore_index=True)
                result_error = result_error.append(
                    {'city_id': city_id, 'is_weekend': 1, 'hour': i, 'error_reason': '数据缺失'}, ignore_index=True)
            else:
                result = result.append({'city_id': city_id, 'is_weekend': 1, 'hour': i, 'xgb_piecewise': -1, 'xgb_min': -1, 'xgb_max': -1,
                                       'cr_piecewise': -1, 'xgb_target': xgb_target, 'xgb_target_min': -1, 'xgb_target_max': -1}, ignore_index=True)
                result_error = result_error.append(
                    {'city_id': city_id, 'is_weekend': 1, 'hour': i, 'error_reason': '拟合失败'}, ignore_index=True)
    return result, result_error


def func_xgb_target(data_xgb_pass_ori, data_fzb_cr_rt, data_xgb_t_last):
    data_xgb_pass_ori['is_weekend'] = data_xgb_pass_ori['dt'].apply(
        lambda x: 1 if datetime.datetime.strptime(x, '%Y-%m-%d').weekday() in [5, 6] else 0)
    data_xgb_pass_ori['dt'] = pd.to_datetime(data_xgb_pass_ori['dt'])
    data_xgb_pass_ori['dt'] = data_xgb_pass_ori['dt'].apply(
        lambda x: x.strftime('%Y-%m-%d'))
    data_xgb_pass_ori['应答率'] = data_xgb_pass_ori['cnt_answer'] / \
        data_xgb_pass_ori['cnt_call']
    data_xgb_pass_ori['成交率'] = data_xgb_pass_ori['cnt_finish'] / \
        data_xgb_pass_ori['cnt_call']
    data_xgb_pass_ori['总需求时长'] = data_xgb_pass_ori['demand_dur']
    data_xgb_pass_ori['总供给时长'] = data_xgb_pass_ori['online_dur']
    data_xgb_pass_ori['供需比'] = data_xgb_pass_ori['origin_xgb']
    data_xgb_pass_ori['供需比分组'] = (data_xgb_pass_ori['供需比']*100//2)*0.02
    data_xgb_pass_ori = data_xgb_pass_ori.dropna(axis=0, how='any')
    # 剔除疫情
    data_xgb_pass = data_xgb_pass_ori[data_xgb_pass_ori.is_tichu == 0]
    # 2.计算最优供需比
    # 2.1构建基础数据
    # 计算供需比分组下的：成交率、样本量
    func = {'成交率': 'mean', 'dt': 'count'}
    data_fit_pass = data_xgb_pass.groupby(
        ['city_id', 'city_name', 'hour', 'is_weekend', '供需比分组']).agg(func).reset_index()
    # 计算服占比
    data_fzb_pass = data_xgb_pass[['dt', 'city_id', 'city_name', 'hour', 'is_weekend', '供需比分组', 'cnt_finish', 'cnt_call', 'cnt_answer',
                                   'online_time', 'online_busy_time']].groupby(['city_id', 'city_name', 'hour', 'is_weekend', '供需比分组']).sum().reset_index()
    data_fzb_pass = data_fzb_pass['online_busy_time'] / \
        data_fzb_pass['online_time']
    # merge
    data_fit_pass = pd.merge(data_fit_pass, data_fzb_pass, left_on=[
                             'city_id', 'city_name', 'hour', 'is_weekend', '供需比分组'], right_index=True, how='left')
    # 修正基础数据
    data_fit_pass = pd.merge(data_fit_pass, data_fzb_cr_rt[['city_id', 'hour', 'is_weekend', 'cr', 'fzb']], left_on=[
                             'city_id', 'hour', 'is_weekend'], on=['city_id', 'hour', 'is_weekend'], how='inner')
    # 服占比修正
    data_fit_pass['服占比修正'] = data_fit_pass.apply(
        lambda x: rt_fzb_modified(x.服占比, x.fzb_lower, x.fzb_upper), axis=1)
    # 供需比约束区间
    data_fit_pass['xgb_target_min'] = data_fit_pass.cr_lower / \
        data_fit_pass['服占比修正']
    data_fit_pass['xgb_target_max'] = data_fit_pass.cr_upper / \
        data_fit_pass['服占比修正']
    data_fit_pass = data_fit_pass.sort_values(
        by=['city_id', 'hour', 'is_weekend', '供需比分组'], ascending=True)
    # 2.3拟合，找拐点
    # top100城市循环，输出step2的最优供需比
    data_result_xgb_pass = pd.DataFrame(columns=['city_id', 'city_name', 'hour', 'is_weekend', 'xgb_target',
                                        'xgb_target_min', 'xgb_target_max', 'xgb_min', 'xgb_max', 'xgb_piecewise', 'cr_piecewise'])
    error_list = pd.DataFrame(
        columns=['city_id', 'is_weekend', 'hour', 'error_reason'])
    for city_id in data_fit_pass.city_id.unique():
        data_fit_pass_city = data_fit_pass[data_fit_pass.city_id == city_id]
        data_result_xgb_pass_city, error_list_city = func_result(
            data_fit_pass_city, city_id)

        data_result_xgb_pass = pd.concat(
            [data_result_xgb_pass, data_result_xgb_pass_city], axis=0)
        error_list = pd.concat([error_list, error_list_city], axis=0)
    data_result_xgb_pass['city_id'] = data_result_xgb_pass['city_id'].astype(
        int)
    data_result_xgb_pass['hour'] = data_result_xgb_pass['hour'].astype(int)
    data_result_xgb_pass['is_weekend'] = data_result_xgb_pass['is_weekend'].astype(
        int)
    data_result_xgb_pass = data_result_xgb_pass.sort_values(
        by=['city_id', 'hour', 'is_weekend'], ascending=True)
    # 上月最优供需比
    data_xgb_t_last = data_xgb_t_last.rename(
        columns={'xgb_target_modified': 'xgb_target_modified_last'})
    data_result_xgb_pass = pd.merge(data_result_xgb_pass, data_xgb_t_last[[
                                    'city_id', 'hour', 'is_weekend', 'xgb_target_modified_last']], on=['city_id', 'hour', 'is_weekend'], how='left')
    # 判断疫情天数
    data_xgb_pass_ori_group_temp = data_xgb_pass_ori.groupby(
        ['city_id', 'hour', 'is_weekend']).agg({'is_tichu': 'sum', 'dt': 'count'}).reset_index()
    data_xgb_pass_ori_group_temp['is_tichu_rate'] = data_xgb_pass_ori_group_temp['is_tichu'] / \
        data_xgb_pass_ori_group_temp['dt']
    data_result_xgb_pass = pd.merge(data_result_xgb_pass, data_xgb_pass_ori_group_temp[[
                                    'city_id', 'hour', 'is_weekend', 'is_tichu_rate']], on=['city_id', 'hour', 'is_weekend'], how='left')
    # 如果疫情天数>40%,且上月最优供需比不为空，则取上月最优供需比；否则取本月
    data_result_xgb_pass['xgb_target_modified_temp'] = data_result_xgb_pass.apply(lambda x: x['xgb_target_modified_last'] if (
        (x['is_tichu_rate'] > 0.4) and (x['xgb_target_modified_last'] > 0)) else x['xgb_target'], axis=1)
    # 修正拐点
    # 1.修正0时或23时为-1的部分
    for i in range(0, len(data_result_xgb_pass.city_id.unique())*2):
        # 获取第i个城市+周中、周末的数据
        row = i*24
        # 正序，取第一个不为-1的时段和最优供需比
        firstvalid = -1
        firstvalidindex = -1
        for j in range(0, 24):
            if data_result_xgb_pass.iloc[row+j, 12] != -1:
                firstvalid = data_result_xgb_pass.iloc[row+j, 12]
                firstvalidindex = row+j
                break
        # 倒序，取最后一个不为-1的时段和最优供需比
        lastvalid = -1
        lastvalidindex = -1
        for j in range(23, -1, -1):
            if data_result_xgb_pass.iloc[row+j, 12] != -1:
                lastvalid = data_result_xgb_pass.iloc[row+j, 12]
                lastvalidindex = row+j
                break
        # 正序，将从0开始连续供需比为-1的时段，填充为firstvalid
        for j in range(0, 24):
            if data_result_xgb_pass.iloc[row+j, 12] == -1:
                data_result_xgb_pass.iloc[row+j, 12] = firstvalid
            else:
                break
        # 倒序，将从23开始连续供需比为-1的时段，填充为lastvalid\
        for j in range(23, -1, -1):
            if data_result_xgb_pass.iloc[row+j, 12] == -1:
                data_result_xgb_pass.iloc[row+j, 12] = lastvalid
            else:
                break
    data_result_xgb_pass['xgb_target_modified'] = 0
    # 2,对中间部分的-1，取前后时段最近的不为-1的时段的最优供需比平均值
    for i in range(0, len(data_result_xgb_pass.city_id.unique())*2):
        # 前一个不为-1的最优供需比
        lastestxgb = 0
        # 后一个不为-1的最优供需比
        nextxgb = 0
        row = i*24
        for j in range(0, 24):
            # 如果该时段最优供需比不为-1，修正后最优供需比=原最优供需比，并记录为lastestxgb
            if data_result_xgb_pass.iloc[row+j, 12] != -1:
                lastestxgb = data_result_xgb_pass.iloc[row+j, 12]
                data_result_xgb_pass.iloc[row+j, 13] = lastestxgb
            else:
                hasfind = 0
                # 往后找，取后面时段不为-1的最优供需比为nextxgb
                for k in range(row+j+1, row+24):
                    if (data_result_xgb_pass.iloc[k, 12] != -1):
                        hasfind = 1
                        nextxgb = data_result_xgb_pass.iloc[k, 12]
                        break
                # 如果找到了，则修正后最优供需比=lastestxgb+nextxgb/2
                data_result_xgb_pass.iloc[row+j, 13] = (lastestxgb+nextxgb)/2
    city_name = data_xgb_pass[['city_id', 'city_name']].drop_duplicates()
    data_result_xgb_pass = pd.merge(
        data_result_xgb_pass, city_name, on='city_id', how='left')
    error_list = pd.merge(error_list, city_name, on=['city_id'], how='left')
    return data_result_xgb_pass, error_list


In [ ]:
input=datetime.datetime.strptime(i,'%Y-%m-%d')
date_start_month=datetime.date(input.year,input.month,1)
date_start_xgb_pass=(date_start_month-datetime.timedelta(days=date_start_month.weekday()+168)).strftime('%Y-%m-%d')
date_end_xgb_pass=(date_start_month-datetime.timedelta(days=date_start_month.weekday()+1)).strftime('%Y-%m-%d')
print('input:',input,'输入起始日期',date_start_xgb_pass,'输入结束日期',date_end_xgb_pass)
#读取历史24周数据
df_xgb_pass_or=pd.read_csv('data_xgb_pass.csv')
df_xgb_pass=df_xgb_pass_or[(df_xgb_pass_or['date']>=date_start_xgb_pass)&(df_xgb_pass_or['date']<=date_end_xgb_pass)]
#读取上个月最优供需比
input_last=(date_start_month-datetime.timedelta(days=1)).strftime('%Y-%m-%d')
date_last_month=datetime.date(input_last.year,input_last.month,1)
print('上个月最优供需比日期',date_last_month)
df_xgb_t_last=pd.read_csv('data_xgb_t_last.csv')
df_xgb_t_last=df_xgb_t_last[df_xgb_t_last['date']==date_last_month.strftime('%Y-%m-%d')]
#诊断
df_fzb_cr_rt=pd.read_csv('data_fzb_cr_rt.csv')
df_result_xgb=func_xgb_target(df_xgb_pass,df_xgb_t_last,df_fzb_cr_rt)